In [1]:
import os
%pwd

'/Users/vaidehipawar/Desktop/mlops/TextSummarizer/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/vaidehipawar/Desktop/mlops/TextSummarizer'

In [12]:
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

# Then run your training

## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy:  str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


## Config

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config



In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/vaidehipawar/Desktop/mlops/TextSummarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-25 16:08:53,479: INFO: config: PyTorch version 2.7.1 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
        output_dir=self.config.root_dir, 
        num_train_epochs=self.config.num_train_epochs,  # Use config value
        warmup_steps=self.config.warmup_steps,          # Use config value
        per_device_train_batch_size=self.config.per_device_train_batch_size,
     
        weight_decay=self.config.weight_decay,
        logging_steps=self.config.logging_steps,
        eval_strategy=self.config.eval_strategy,
        eval_steps=self.config.eval_steps,
        save_steps=self.config.save_steps,
        gradient_accumulation_steps=self.config.gradient_accumulation_steps
    ) 
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))




In [9]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached accelerate-1.8.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
Using cached accelerate-1.8.1-py3-none-any.whl (365 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    import traceback
    print("Full traceback:")
    traceback.print_exc()

[2025-06-25 16:15:44,760: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-25 16:15:44,762: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 16:15:44,763: INFO: common: created directory at: artifacts]
[2025-06-25 16:15:44,763: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/ws/xgj4142s6p921_2v03p33lvw0000gn/T/ipykernel_54425/403731106.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


Full traceback:


Traceback (most recent call last):
  File "/var/folders/ws/xgj4142s6p921_2v03p33lvw0000gn/T/ipykernel_54425/14399852.py", line 5, in <module>
    model_trainer.train()
  File "/var/folders/ws/xgj4142s6p921_2v03p33lvw0000gn/T/ipykernel_54425/403731106.py", line 32, in train
    trainer.train()
  File "/Users/vaidehipawar/Desktop/mlops/TextSummarizer/venv/lib/python3.10/site-packages/transformers/trainer.py", line 2240, in train
    return inner_training_loop(
  File "/Users/vaidehipawar/Desktop/mlops/TextSummarizer/venv/lib/python3.10/site-packages/transformers/trainer.py", line 2555, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs, num_items_in_batch)
  File "/Users/vaidehipawar/Desktop/mlops/TextSummarizer/venv/lib/python3.10/site-packages/transformers/trainer.py", line 3745, in training_step
    loss = self.compute_loss(model, inputs, num_items_in_batch=num_items_in_batch)
  File "/Users/vaidehipawar/Desktop/mlops/TextSummarizer/venv/lib/python3.10/site-pa